# 1. Filter and tag raw PQ data as unanswered 

We wanted to analyse how many Parlimentary written questions (WQs) could not be answered because the department did not have the requisite data, and what those questions were about.

We have first obtained text for all the WQs in the current Parliament from TheyWorkForYou. 

In this notebook:

- we identify common strings to indicate quantitative questions - these are particularly of interest to us as they require data to answer
- we identify common strings that indicate questions that could not be answered, because the data is not held or it is not possible to derive the answer from the data.

We can then review these questions manually, to see which of these questions were truly unanswered, and what themes recurred for each Department.

NB: The text of Hansard answers remains under [Parliamentary copyright](https://www.parliament.uk/site-information/copyright-parliament/).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Combine data from all years

We have previously scraped all written questions answered, from TheyWorkForYou. Load these into a single dataframe.

In [4]:
dfs = []
for year in range(2020, 2024):
    df = pd.read_csv("data/raw_answers/output_%s.csv" % year)
    dfs.append(df)
df = pd.concat(dfs)

In [5]:
df.replace(np.nan, '', regex=True, inplace=True)
df.date_submitted = pd.to_datetime(df.date_submitted, format="mixed")
df.date_answered = pd.to_datetime(df.date_answered, format="mixed")
pd.options.display.max_colwidth = 50
print(len(df), "questions in our dataset")

203938 questions in our dataset


## Identify quantitative questions

Written questions can seek quantitative or non-quantitative information. Common phrases for quantitative questions include "how many", "how much", "what estimate", etc.

About one-sixth of questions contain these phrases, so are probably or definitely quantitative in nature. Note that this is not definitive, because some quantitative questions also contain other phrases. 

Also, the phrase 'what assessment' is used sometimes in quantitative questions, although it's frequently used in non-quantitative questions too, so isn't useful for filtering.

In [21]:
s_quantitative = 'how many|how much|what estimate|what proportion|what the average'
df['is_quantitative'] = df.question_text.str.contains(s_quantitative, case=False, regex=True)
df_quantitative = df[df.is_quantitative]
print(len(df_quantitative), 'quantitative questions submitted')
print("%.1f%% of the total" % (len(df_quantitative) / len(df) * 100))

37106 quantitative questions submitted
18.2% of the total


## Extract questions that could not be answered

Filter the answer text for common ways of saying "we do not have the data" - see the regex below.

Note that filtered questions need manual review, because this method can produce false positives, as the answering department sometimes says:

- it doesn't hold the data itself but refers the questioner to a different authority that does
- it does not have (e.g.) data by constituency but provides data by local authority
- it uses the phrase in another context entirely

However, none of these is that common, and this method is a useful benchmark.

Note also that sometimes answering departments will simply not answer the question, but without saying why, so this method also produces false negatives - and there's not much we can do about it. This is not especially common though.

In [24]:
# Flag questions that contain strings suggesting data is not held, or not possible to extract.
s_notheld = 'do(es)? not (currently|routinely)?\s?(hold|collect|record|publish)'
s_notheld += '|(is|are) not (currently|routinely)?\s?(available|recorded|held|collected|published|kept)'
s_notheld += '|(is|are) (currently)?\s?unavailable'
s_notheld += '|unable to answer|not possible to provide|disproportionate cost|no(t)? central(ly)?'
df['contains_not_held_string'] = df.answer_text.str.contains(s_notheld, case=False, regex=True)
df_not_held = df[df.contains_not_held_string]
print(len(df_not_held), "questions contain 'not held' string")
print("That's %.1f%% of the total" % (len(df_not_held)/len(df) * 100))

# Flag questions that both contain those strings, and are quantitative in nature.
df['contains_not_held_string_and_isquant'] = df.is_quantitative & df.contains_not_held_string
print(len(df[df.contains_not_held_string_and_isquant]), "questions contain 'not held' and appear quantitative")
print("That's %.1f%% of the total" % (len(df[df.contains_not_held_string_and_isquant])/len(df) * 100))

# Flag questions that suggest it is not possible to answer the question. Note that many of these
# are not quantitative questions, so record this separately.
# Some departments seem to use this phrase a lot, some not at all.
# NB Some departments seem to use the following 'not possible' strings a lot - some not at all.
s_notpossible = 'not possible to answer|not be possible to answer'
df['contains_not_possible_string'] = df.answer_text.str.contains(s_notpossible, case=False, regex=True)
df_not_possible = df[df.contains_not_possible_string]
print(len(df_not_possible), 'were flagged as "not possible" to answer')
print("That's %.1f%% of the total" % (len(df_not_possible)/len(df) * 100))

/var/folders/h5/r_p1fbvx13g1hf0c15pjx60h0000gn/T/ipykernel_28667/592036416.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['contains_not_held_string'] = df.answer_text.str.contains(s_notheld, case=False, regex=True)


14377 questions contain 'not held' string
That's 7.0% of the total
9615 questions contain 'not held' and appear quantitative
That's 4.7% of the total
1526 were flagged as "not possible" to answer
That's 0.7% of the total


In [23]:
df.to_csv('./data/output/questions_with_flags.csv', index=False)

## Do a sense check

Let's have a quick squiz at a small dataset - questions to Church Commissioners - to see if the method seems to make sense.

This seems reasonable - not many false positives in here. It also quickly shows us the themes that are being asked about that can't be answered - chiefly about spending, particularly local breakdowns of spending; and updates on recruitment.

In [14]:

pd.set_option('display.max_colwidth', None)
df[(df.department == "Church Commissioners") & df.contains_not_held_string_and_isquant][['question_text', 'answer_text']]

,question_text,answer_text
37121,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, how much the Church of England spent on the maintenance and repair of its parish churches in the financial year 2019-20.\n\n","A comprehensive record of maintenance and repair spend is not held centrally, as all parishes operate as independent entities. However it is estimated that parish communities spent at least £130m a year pre-pandemic on church building projects, including repair and maintenance, conservation of collections such as manuscripts, silver and textiles, improvements to accessibility and environmental sustainability, and inclusion of community facilities such as toilets and kitchens. Specialist building firms, craftspeople, archaeologists, conservators and others rely on churches to provide them with work and a training ground for passing their skills on to apprentices and trainees.The vast majority of this money is raised locally and we are grateful to national bodies such as the National Lottery Heritage Fund, National Churches Trust, Wolfson Foundation and Pilgrim Trust, who continue to provide essential grants. The overall maintenance deficit continues to grow since the ending of the dedicated places of worship repair scheme in 2017."
38716,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, how many clergy have taken their own lives in the last five years.\n\n","Records are not held centrally on the causes of death of clergy. The Church of England provides resources for those experiencing stress and signposts to other resources from charities and organisations able to help. Some more information is available here: Supporting clergy health and wellbeing | The Church of England As chief pastors in their dioceses, diocesan bishops ensure pastoral care is available to clergy and is provided when needed.The Church of England is committed to supporting those individuals and organisations working to help those at risk of self-harm and suicide. At a parish level many clergy play a key role themselves in providing counselling, comfort and support to those at risk. All suicides are tragedies, and whilst causes will vary, the death of any member of the clergy through suicide will be a matter of profound concern and deep sadness for all in the church, their community and all who have a responsibility for their pastoral care."
40309,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, how much has been paid in (a) tax rebates, including of VAT on repairs and (b) Gift Aid to Church of England bodies including (i) dioceses, (ii) parochial church councils, (iii) cathedrals, (iv) the Church Commissioners and (v) the Churches Conservation Trust in each of the last five years.\n\n","Dioceses, parochial church councils and cathedrals:Government (DCMS) figures for grants equal to the value of VAT for works to listed places of worship over the past five years are below. These are grants to the value of VAT, not rebates. The Church Commissioners do not hold these figures broken down by Church of England diocese, parochial church council or cathedral:2016 £20,689,034.002017 £23,839,469.002018 £24,773,038.002019 £24,522,362.002020 £24,665,225.00This website, maintained by DCMS, contains details of the Listed Places of Worship Scheme and recent total disbursement: http://www.lpwscheme.org.uk/Like all bodies that have charitable status, Gift Aid on donations can be claimed by parochial church councils, dioceses and cathedrals. The funding model of the Church of England means that the vast majority of donations from worshippers, visitors and other supporters are made to parochial church councils and to cathedral chapters. The Church Commissioners do not keep a record of Gift Aid receipts by dioceses, but the amount claimed will be modest.Parochial Church Council Gift Aid receipts for the past five years are:2016 £91.9m2017 £94.5m2018 £94.3m2019 £9

What about questions that do seem to have been answered OK - or that weren't quantitative? Can we spot false negatives? 

Again this seems reasonable, not lots of false negatives leaping out of the page.

In [15]:
df[(df.department == "Church Commissioners") & ~df.contains_not_held_string_and_isquant][['question_text', 'answer_text']].sample(n=20)

,question_text,answer_text
24239,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, whether any properties owned by the Church of England are being used under the Homes for Ukraine scheme.\n\n","The Church of England does not keep a central database of those sponsoring Ukrainian refugees, but guidance for parishes and clergy on the processes involved is available from the National Church Institutions and published here: https://www.churchofengland.org/sites/default/files/2022-04/Ukraine%20toolkit%20draft%208.pdfAnecdotal evidence from dioceses suggests that expressions of interest number in the hundreds, including those in clergy accommodation, and that matches and arrivals have been steadily increasing. The Church Commissioners are aware of four bishops who have successfully sponsored refugees, with several others still waiting either to be matched or for their application to be completed. The Church of England Pensions Board and others have also issued guidance and are aware of residents taking up the opportunity to host Ukrainians in those properties"
35761,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, with reference to Review of Lowest Income Communities and Strategic Development Funding, published in March 2022, what steps the Church Commissioners have taken to implement new governance arrangements to ensure that Lowest Incomes Communities Funding is distributed to dioceses by the Archbishops’ Council is accounted for (a) effectively and (b) transparently.\n\n","The Independent Review of Lowest Income Communities Funding and Strategic Development Funding led by Sir Robert Chote was published in March 2022 and can be read here: https://www.churchofengland.org/sites/default/files/2022-03/irls-final-report-2.pdfIn respect of improving governance arrangements of Lowest Income Communities Funding (LInC) the Review recommended:Recommendation 2 - ""Refocusing annual returns … towards more strategic reporting"".Recommendation 3 - asked staff to ""identify and promote best practice … perhaps convening a learning symposium"".Recommendation 5 – asked staff to ""communicate more clearly the extent to which LInC funding is sustaining ministry"".The action taken so far by the Archbishops' Council's Vision and Strategy team in response to those recommendations has been to:Raise the profile of LInC funding within ongoing strategic conversations between its staff and diocesan teams, as part of dioceses' strategic development processes, with the aim of improving its effectiveness.Ask dioceses to give information about the intended outcomes from their planned use of LInC funding, together with how they will be measured, as part of their annual return on LInC funding for 2022.Plan a learning event to bring together diocesan representatives to share and develop their approaches on the use of LInC funding, aiming to improve the effectiveness, accountability and transparency of LInC funding.The Independent Review also made further suggestions in Recommendation 1: ""encouraging dioceses to use LInC funding more for clergy transitions"" and in Recommendation 4: ""maintaining current levels of LInC funding"". While not related directly to governance, these may also feature in the learning event and in ongoing conversations with dioceses.From the start of 2023, the new Strategic Mission and Ministry Investment Board (SMMIB), a committee of the Archbishops' Council, has replaced the previous Strategic Investment Board and Strategic Ministry Board. It distributes and monitors funds made available by the Church Commissioners and Archbishops' Council via LInC, SDF, and other similar funds.The Boards of the Archbishops’ Council and the Church Commissioners are regularly updated by the SMMIB about funding decisions and projects supported by these funds. Members of the Archbishops’ Council and Church Commissioners Boards are part of the SMMIB and help monitor funding decisions an

In [17]:
df[(df.department == "Church Commissioners") & df.is_quantitative & ~df.contains_not_held_string_and_isquant][['question_text', 'answer_text']]

,question_text,answer_text
4169,"To ask the right hon. Member for South West Bedfordshire, representing the Church Commissioners, what estimate the Church of England has made of the number of church-based youth clubs; and what assessment he has made of effect of those clubs on the wellbeing of the young people that use them.\n\n","The Church of England employs 2,670 children and youth workers in its parishes, and there are 80,000 volunteers across the church who are engaged in projects supporting children and young people. Projects can range from holiday clubs, messy church, sports ministry and creative play.The Church of England commissioned a research project assessing how best to approach building confidence, engagement and belonging, creating a supportive and inclusive environment in 2016 which has formed the basis of its approach. The report can be read here: https://www.churchofengland.org/sites/default/files/2017-11/Rooted%20in%20the%20Church%20Summary%20Report.pdf"
17241,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, what the average level of compensation paid for tenants’ improvements following the expiry of farm tenancies let by the Church Commissioners was in each of the last five years, ending with 2020.\n\n",Compensation payments made by the Commissioners to their tenants are of a commercially sensitive nature.The Church Commissioners discuss land and environmental improvement works with their agricultural tenants on an ongoing basis with a view to supporting enhancements to holdings. End of tenancy payments are made as appropriate to the scale and value of such improvement made.
17242,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, other than in cases where a tenancy is replacing a previous tenancy let under the Agricultural Holdings Act 1986 how many tenancies have been let (a) for an initial fixed term of five years or less (b) for an initial fixed term of five years or more in each of the last five years, ending with 2020.\n\n","The Church Commissioners’ and their managing agents review the term of each new farm letting and all tenancy renewals on a case by case basis. In the period 2016 to 2020, the number of holdings let by the Church Commissioners on a Farm Business Tenancy (on an area of more than 50 acres) for a term under five years is 64, with a further 30 new tenancies offered for a term of 5 years or more. Each holding is assessed at the time of letting, and other factors considered when determining term length."
19232,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, what the average length of term agreed on farm business tenancies let by the Church Commissioners, other than in cases where a tenancy is replacing a previous tenancy let under the Agricultural Holdings Act 1986, was in each of the last five years to 2020.\n\n","In the last five years, ending with 2020, the average length of the term granted for new farm business tenancies over 50 acres across the Commissioners’ rural portfolio was:20164.120174.920183.620193.520202.3Such averages do not reflect the size of the holdings which were let during this period or the circumstances and rationale for each new farm letting."
46377,"To ask the Member for South West Bedfordshire, representing the Church Commissioners, how many services the Church of England has conducted specifically for the purposes of Baptising people seeking asylum in each of the last five years.\n\n","Data are not kept on the nationality or migration status of those who seek Baptism. The content of the rite of Baptism remains the same for all.Clergy are required by canon law to ensure that all who come to Baptism as adults are given appropriate teaching on the tenets of the faith and in encouragement to private spiritual devotion. Therefore, the preparation process may require specific, lengthier input and instruction of some groups of candidates before the clergype